- #### Transformaciones
    - ##### Windows

In [1]:
import org.apache.spark.sql.{SparkSession, DataFrame}

val spark = SparkSession.builder
        .appName("sesion_1")
        .master("local[*]")
        .getOrCreate()
val sc = spark.sparkContext

sc = org.apache.spark.SparkContext@7d62aa27


spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@31aaaf31


org.apache.spark.SparkContext@7d62aa27

In [2]:
sc.uiWebUrl

Some(http://23LAP5CD20860NP.indra.es:4040)

In [3]:
def readCsv(path: String): DataFrame = {
    spark.read
        .option("header","true")
        .option("delimiter",",")
        .option("inferSchema","false")
        .csv(path)
}

val BasePath = "../../resources/data/csv/"
val contractsDf = readCsv(BasePath + "contracts.csv")
contractsDf.show()

+-------+-----------+------------+----------+------+
|cod_iuc|cod_titular|cod_producto|  fec_alta|activo|
+-------+-----------+------------+----------+------+
|  30000|      00006|         100|2012-05-01|  true|
|  30001|      00006|         200|2014-05-01|  true|
|  30002|      00006|         300|2006-02-01| false|
|  30003|      00006|         150|2012-05-01|  true|
|  30002|      00005|         300|2012-05-01|  true|
|  30004|      00006|         400|2012-05-01| false|
|  30005|      00006|         500|2012-05-01|  true|
|  30006|      00006|         600|2012-05-01| false|
|  30003|      00003|         150|2019-10-14|  true|
|  30007|      00006|         700|2014-02-01| false|
|  30008|      00006|         800|2012-05-01|  true|
|  30009|      00006|         900|2015-09-01|  true|
|  30009|      00002|         900|2009-10-01|  true|
|  30010|      00006|        1000|2014-02-01|  true|
|  30003|      00002|         150|2018-09-18|  true|
|  30011|      00003|        1100|2018-10-01| 

contractsDf = [cod_iuc: string, cod_titular: string ... 3 more fields]


readCsv: (path: String)org.apache.spark.sql.DataFrame
BasePath: String = ../../resources/data/csv/


[cod_iuc: string, cod_titular: string ... 3 more fields]

In [ ]:
// Windows
    // Agregacion -> max, min, count, avg, mean, sum, ...
    // Ranking -> row_number, rank, dense_rank, ...
    // lag lead

import org.apache.spark.sql.expressions.{Window, WindowSpec}
import org.apache.spark.sql.{functions => f}

contractsDf.orderBy(f.col("cod_iuc")).show()

val window_1: WindowSpec = Window.partitionBy(f.col("cod_iuc"))
val window_2: WindowSpec = Window.partitionBy(f.col("cod_iuc")).orderBy(f.col("fec_alta").asc)


contractsDf.select(
    contractsDf.columns.map(f.col) :+
    f.count("*").over(window_1).alias("count_w") :+
    f.max(f.col("fec_alta")).over(window_2).alias("max_fec_alta_w") :+
    f.sum(f.col("cod_producto")).over(window_2).alias("sum_cod_producto_w") :_*
).show()

In [ ]:
contractsDf.orderBy(f.col("cod_iuc")).show()

val window_3: WindowSpec = Window.partitionBy(f.col("cod_iuc")).orderBy(f.col("cod_titular").asc)

contractsDf.select(
    contractsDf.columns.map(f.col) :+
    f.row_number().over(window_3).alias("row_number") :+
    f.rank().over(window_3).alias("rank") :+
    f.dense_rank().over(window_3).alias("dense_rank") :_*
).show(100)

In [ ]:
contractsDf.orderBy(f.col("cod_iuc")).show()

val window_4: WindowSpec = Window.partitionBy(f.col("cod_iuc")).orderBy(f.col("fec_alta").desc)

contractsDf.select(
    contractsDf.columns.map(f.col) :+
    f.lag(f.col("fec_alta"), 1, "1970-01-01").over(window_4).alias("lag") :+
    f.lead(f.col("fec_alta"), 1, "1970-01-01").over(window_4).alias("lead") :_*
).show(100)